# RDKit C++ api basic examples with cling and xeus-cling kernel



- CERN's cling C++ interpreter https://github.com/root-project/cling
- QuantStack xeus-cling jupyter notebook kernel https://github.com/QuantStack/xeus-cling

## Add RDKit  to include and library paths

In [1]:
#pragma cling add_library_path("/rdkit/lib/")
#pragma cling add_include_path("/rdkit/Code/")

## Load methotrexate from SMILES and count its atoms


In [2]:
#include <GraphMol/GraphMol.h>
#include <GraphMol/SmilesParse/SmilesParse.h>

#pragma cling load("libRDKitSmilesParse.so")
#pragma cling load("libRDKitGraphMol.so")

std::string methotrexate_smi = "CN(CC1=CN=C2C(=N1)C(=NC(=N2)N)N)C3=CC=C(C=C3)C(=O)NC(CCC(=O)O)C(=O)O";
RDKit::ROMol *romol = RDKit::SmilesToMol(methotrexate_smi);
std::cout << romol->getNumAtoms() << " atoms" << std::endl;

33 atoms


## Calc some properties

In [3]:
#include <GraphMol/Descriptors/MolDescriptors.h>
#pragma cling load("libRDKitDescriptors.so")

double logp = RDKit::Descriptors::calcClogP(*romol);
std::cout << "logP: " << logp << std::endl;

double e_mwt = RDKit::Descriptors::calcExactMW(*romol);
std::cout << "molecular weight: " << e_mwt << std::endl;


logP: 0.2684
molecular weight: 454.171


## 2D coord generation and depiction

In [4]:
#include <GraphMol/MolDraw2D/MolDraw2DSVG.h>
#include <GraphMol/Depictor/RDDepictor.h>
#include <fstream>
#include <string>
#include "xcpp/xdisplay.hpp"

#pragma cling load("libRDKitMolDraw2D.so")
#pragma cling load("libRDKitDepictor.so")

namespace ht
{
    struct html
    {   
        inline html(const std::string& content)
        {
            m_content = content;
        }
        std::string m_content;
    };

    xeus::xjson mime_bundle_repr(const html& a)
    {
        auto bundle = xeus::xjson::object();
        bundle["text/html"] = a.m_content;
        return bundle;
    }
}

In [5]:
// generate the 2D coordinates:
RDDepict::compute2DCoords(*romol);

// // generate SVG as string
RDKit::MolDraw2DSVG drawer(400, 400);
drawer.drawMolecule(*romol);
drawer.finishDrawing();
std::string svgs = drawer.getDrawingText();

// show the molecule
ht::html mol_svg(svgs);
mol_svg

<?xml version='1.0' encoding='iso-8859-1'?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 N 
 N 
 N 
 N 
 N 
 NH 2 
 NH 2 
 O 
 NH 
 O 
 HO 
 O 
 HO

## Morgan fingerprint and Tanimoto similarity

In [6]:
#include <GraphMol/Fingerprints/Fingerprints.h>
#include <GraphMol/Fingerprints/MorganFingerprints.h>
#include <DataStructs/ExplicitBitVect.h>
#include <DataStructs/BitOps.h>

#pragma cling load("libRDKitFingerprints.so")
#pragma cling load("libRDKitDataStructs.so")


std::string aspirin_str = "CC(=O)OC1=CC=CC=C1C(=O)O";
std::string paracetamol_str = "CC(=O)NC1=CC=C(C=C1)O";

RDKit::ROMol *ro_aspirin = RDKit::SmilesToMol(aspirin_str);
RDKit::ROMol *ro_para = RDKit::SmilesToMol(paracetamol_str);

// unhashed fps
RDKit::SparseIntVect<boost::uint32_t> *fp_aspirin, *fp_paracetamol;

fp_aspirin = RDKit::MorganFingerprints::getFingerprint(*ro_aspirin, 2);
fp_paracetamol = RDKit::MorganFingerprints::getFingerprint(*ro_para, 2);

double tani = TanimotoSimilarity(*fp_aspirin, *fp_paracetamol);
std::cout << "Similarity between aspirin and paracetamol: " << tani << std::endl;


Similarity between aspirin and paracetamol: 0.27451


## Substructure search

In [7]:
#include <GraphMol/Substruct/SubstructUtils.h>
#pragma cling load("libRDKitSubstructMatch.so")

std::shared_ptr<RDKit::RWMol> mol_ss(RDKit::SmilesToMol("CC[C@H](F)Cl"));
std::shared_ptr<RDKit::ROMol> patt(RDKit::SmartsToMol("C[C@H](F)Cl"));

RDKit::MatchVectType res;

if( RDKit::SubstructMatch(*mol_ss , *patt , res) ) {
    std::cout << "SMARTS 1 match" << std::endl;
  } else {
    std::cout << "Not SMARTS 1 match" << std::endl;
}

SMARTS 1 match
